In [1]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[0.0227],
        [0.0413]], grad_fn=<AddmmBackward0>)

In [2]:
net[2].state_dict()

OrderedDict([('weight',
              tensor([[-0.0652, -0.1177, -0.0410, -0.0027,  0.2845,  0.1577,  0.2618,  0.1059]])),
             ('bias', tensor([0.1084]))])

In [3]:
net.state_dict()

OrderedDict([('0.weight',
              tensor([[ 0.3925,  0.2413,  0.4427,  0.1170],
                      [-0.3317,  0.0834, -0.2805,  0.1945],
                      [ 0.2299, -0.0131, -0.2780,  0.4961],
                      [ 0.3126, -0.4978,  0.1365,  0.0361],
                      [ 0.2307, -0.2522, -0.3441, -0.3377],
                      [-0.0283,  0.3549, -0.0448, -0.4416],
                      [-0.4109, -0.4954, -0.0608, -0.0057],
                      [-0.0985,  0.4469, -0.3975, -0.4240]])),
             ('0.bias',
              tensor([ 0.3752,  0.0534, -0.4601, -0.0487, -0.1248, -0.4993, -0.1612, -0.3356])),
             ('2.weight',
              tensor([[-0.0652, -0.1177, -0.0410, -0.0027,  0.2845,  0.1577,  0.2618,  0.1059]])),
             ('2.bias', tensor([0.1084]))])

In [4]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([0.1084], requires_grad=True)
tensor([0.1084])


In [6]:
net[2].weight.grad is None

True

In [7]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [8]:
net.state_dict()['2.bias'].data

tensor([0.1084])

In [10]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[-0.3353],
        [-0.3355]], grad_fn=<AddmmBackward0>)

In [11]:
rgnet

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)

In [12]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)

net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([ 0.0081, -0.0036, -0.0084, -0.0068]), tensor(0.))

In [13]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)

net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [14]:
def xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_normal_(m.weight)

def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([ 0.1609,  0.0703, -0.3576,  0.0965])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [17]:
def my_init(m):
    if type(m) == nn.Linear:
        print('Init', *[(name, param.shape) for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net.state_dict()

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


OrderedDict([('0.weight',
              tensor([[ 0.0000, -5.0638, -0.0000, -0.0000],
                      [ 0.0000,  0.0000, -0.0000, -0.0000],
                      [ 0.0000,  9.5768,  5.3238, -8.9527],
                      [-5.3803, -8.2823,  0.0000, -0.0000],
                      [ 6.3907, -0.0000,  5.7095,  0.0000],
                      [-0.0000,  0.0000, -6.3593, -0.0000],
                      [-8.8309,  7.3452, -0.0000,  6.6894],
                      [-0.0000, -6.5660, -6.7478, -6.5081]])),
             ('0.bias', tensor([0., 0., 0., 0., 0., 0., 0., 0.])),
             ('2.weight',
              tensor([[-6.9199,  7.2121, -0.0000,  0.0000, -9.9608,  0.0000,  0.0000, -0.0000]])),
             ('2.bias', tensor([0.]))])

In [18]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000, -4.0638,  1.0000,  1.0000])

In [19]:
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), shared, nn.ReLU(), shared, nn.ReLU(), nn.Linear(8, 1))
net(X)
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])
